In [ ]:
# Must log in in ArcGIS Pro to import module
import arcpy, re
from arcgis.gis import GIS

In [ ]:
# Extract raw text from tagged HTML string
def clean_html(html):
    rule = re.compile('<.*?>')
    text = re.sub(rule, '', html)
    return text

In [ ]:
# Filling default text when string is null
def write_missing_text(text):
    if text is None:
        return "This product, data, and/or tool is for informational purposes only and may not have been prepared for or be suitable for legal, engineering, or surveying purposes. It does not represent an on-the-ground survey and represents only the approximate relative location of property boundaries. Mapping may not reflect on the ground conditions. The Houston-Galveston Area Council makes no further claims as to the accuracy or reliability of the data, and neither assumes, nor will accept liability for their use. Use or reproduction of this data without acknowledging the Houston-Galveston Area Council as the source is strictly prohibited."
    else:
        return text

In [ ]:
# Log on to ArcGIS Online and initialize
gis = GIS("https://h-gac.maps.arcgis.com", username="username", password="P@$$w0rd")
project = arcpy.mp.ArcGISProject(r"\\Hgac.net\FileShare\ArcGIS\DataSvcs\GIS\Open_Data\Open_Data.aprx")
maps = project.listMaps()

In [ ]:
# Add feature layer to ArcGIS Online
for map in maps:
    if map.name not in ["Boundaries", "Land Cover"]:
        layers = map.listLayers()
        count = 0

        for layer in layers:
            layer_name = r"\\Hgac.net\FileShare\ArcGIS\DataSvcs\GIS\Open_Data\Global_SDE.sde\Global.GLOBAL_ADMIN." + layer.name
            layer_metadata = arcpy.metadata.Metadata(layer_name)
            
            description = clean_html(layer_metadata.description)
            url = "http://gis.h-gac.com/arcgis/rest/services/Open_Data/{}/MapServer/{}".format(map.name, str(count))
            tags = "Open Data, " + layer_metadata.tags
            license_info = write_missing_text(layer_metadata.accessConstraints)

            layer_properties = {
                'description': description,
                'title': layer,
                'url': url,
                'tags': tags,
                'snippet': layer_metadata.summary,
                'accessInformation': layer_metadata.credits,
                'licenseInfo': license_info,
                'type': "Feature Service",
                'access': "public"
            }

            agol_layer = gis.content.add(layer_properties, folder="Open_Data")

            count += 1
            
        else:
            continue